<a href="https://colab.research.google.com/github/samashami/BIG-DATA/blob/main/Lum_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 1: Install & Import Libraries

In [ ]:
!pip install ezdxf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.7 MB/s eta 0:00:00


In [ ]:
import ezdxf
import os

Step 2: Load and Inspect the DXF File

In [ ]:
# Define the path to the uploaded DXF file
dxf_file_path = "/content/drive/My Drive/e5.dxf"

# Load the DXF file
try:
    doc = ezdxf.readfile(dxf_file_path)
    print("DXF file successfully loaded.")
except Exception as e:
    print(f"Error loading DXF file: {e}")

# Get the model space, where drawing entities are stored
msp = doc.modelspace()

# Print a summary of all entities in the DXF file
for entity in msp:
    print(f"Type: {entity.dxftype()}, Layer: {entity.dxf.layer}")

DXF file successfully loaded.
Type: POLYLINE, Layer: 0
Type: CIRCLE, Layer: 0
Type: CIRCLE, Layer: 0
Type: CIRCLE, Layer: 0
Type: CIRCLE, Layer: 0
Type: CIRCLE, Layer: 0
Type: CIRCLE, Layer: 0


Step 3: Extract Dimensions and Properties

In [ ]:
# Helper function to calculate distance between two points
def calculate_distance(point1, point2):
    return ((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2) ** 0.5

# Initialize variables to store the rectangle's coordinates
rect_vertices = []

# Extract rectangle dimensions and coordinates
for entity in msp:
    if entity.dxftype() == 'POLYLINE':
        # Directly access vertices if they are stored as a list
        rect_vertices = [vertex.dxf.location for vertex in entity.vertices]
        print("Vertices of the rectangle:", rect_vertices)

# Calculate width and height of the rectangle
if len(rect_vertices) >= 4:  # Check for at least four corners (rectangular)
    # Assuming vertices are listed sequentially around the shape
    width = calculate_distance(rect_vertices[0], rect_vertices[1])
    height = calculate_distance(rect_vertices[1], rect_vertices[2])
    print(f"Width: {width}, Height: {height}")
else:
    print("Not enough vertices to determine dimensions.")

Vertices of the rectangle: [Vec3(0.0, 0.0, 0.0), Vec3(1202.5, 0.0, 0.0), Vec3(1202.5, 1005.000000000001, 0.0), Vec3(0.0, 1005.000000000001, 0.0)]
Width: 1202.5, Height: 1005.000000000001


Step 4: Adjusting Drill Hole Data Extraction

In [ ]:
# Bottom-right corner of the rectangle
bottom_right_x = max(vertex.x for vertex in rect_vertices)
bottom_right_y = min(vertex.y for vertex in rect_vertices)

# Drill hole information extraction with relative coordinates
for entity in msp:
    if entity.dxftype() == 'CIRCLE':
        center = entity.dxf.center
        radius = entity.dxf.radius
        diameter = 2 * radius
        # Calculate relative coordinates from the bottom-right corner
        rel_x = bottom_right_x - center.x
        rel_y = bottom_right_y - center.y
        print(f"Drill Hole: Center = ({center.x}, {center.y}), Diameter = {diameter} mm, Relative Position = ({rel_x}, {rel_y})")

Drill Hole: Center = (1052.5, 840.0000000000005), Diameter = 25.0 mm, Relative Position = (150.0, -840.0000000000005)
Drill Hole: Center = (601.2499999999997, 979.9999999999997), Diameter = 25.0 mm, Relative Position = (601.2500000000003, -979.9999999999997)
Drill Hole: Center = (149.9999999999998, 840.0000000000005), Diameter = 25.0 mm, Relative Position = (1052.5000000000002, -840.0000000000005)
Drill Hole: Center = (149.9999999999998, 979.9999999999997), Diameter = 25.0 mm, Relative Position = (1052.5000000000002, -979.9999999999997)
Drill Hole: Center = (601.2499999999997, 840.0000000000003), Diameter = 25.0 mm, Relative Position = (601.2500000000003, -840.0000000000003)
Drill Hole: Center = (1052.5, 979.9999999999998), Diameter = 25.0 mm, Relative Position = (150.0, -979.9999999999998)


Step 5: Modify Colors & Layers

In [ ]:
# Change the layer and color of the drill holes
for entity in msp:
    if entity.dxftype() == 'CIRCLE':
        entity.dxf.layer = 'ModifiedLayer'  # Change to a new layer name
        entity.dxf.color = 4  # Setting the color to a predefined number (4 - Cyan in standard AutoCAD colors)

# Save the modified DXF
modified_dxf_path = "/content/drive/My Drive/modified_e5.dxf"
doc.saveas(modified_dxf_path)
print(f"Modified DXF file saved as {modified_dxf_path}")

Modified DXF file saved as /content/drive/My Drive/modified_e5.dxf


Step 6: Add Dimensions and Annotations

In [ ]:
import ezdxf

# Load the modified DXF file
doc = ezdxf.readfile("/content/drive/My Drive/modified_e5.dxf")
msp = doc.modelspace()

# Add dimensions for the rectangle (assuming this part is already correctly implemented)

# Adding diameter dimensions to each drill hole
for entity in msp.query('CIRCLE'):
    center = entity.dxf.center
    radius = entity.dxf.radius
    # Place the dimension slightly offset from the center
    # Here we need to create a dimension entity manually since add_diameter_dim may not work as expected
    leader_offset = 1.5 * radius  # Leader line offset from the center
    dimension_line_position = (center.x + leader_offset, center.y + leader_offset)
    msp.add_circle(center=center, radius=radius, dxfattribs={'color': 4})
    # Use basic dimensioning if specific diameter dimensioning is problematic
    msp.add_linear_dim(
        base=dimension_line_position,
        p1=(center.x - radius, center.y),
        p2=(center.x + radius, center.y),
        angle=0
    ).render()

# Save the updated DXF with dimensions
doc.saveas("/content/drive/My Drive/annotated_e5.dxf")
print("Annotated DXF file saved.")

Annotated DXF file saved.


In [ ]:
import ezdxf

def print_dxf_details(file_path):
    doc = ezdxf.readfile(file_path)
    msp = doc.modelspace()
    print(f"Details for {file_path}:")
    for entity in msp:
        # Use hasattr() to check if the 'color' attribute exists
        color = entity.dxf.color if hasattr(entity.dxf, 'color') else 'Default'
        print(f"Type: {entity.dxftype()}, Layer: {entity.dxf.layer}, Color: {color}")

    # Check for dimensions if present
    dimensions = msp.query('DIMENSION')
    if dimensions:
        for dim in dimensions:
            print(f"Dimension Type: {dim.dxftype()}, Layer: {dim.dxf.layer}")
    else:
        print("No dimensions found.")

# List of file paths
file_paths = ["/content/drive/My Drive/e5.dxf", "/content/drive/My Drive/modified_e5.dxf", "/content/drive/My Drive/annotated_e5.dxf"]
for path in file_paths:
    print_dxf_details(path)

file_paths = ["/content/drive/My Drive/e5.dxf", "/content/drive/My Drive/modified_e5.dxf", "/content/drive/My Drive/annotated_e5.dxf"]
for path in file_paths:
    print_dxf_details(path)

Details for /content/drive/My Drive/e5.dxf:
Type: POLYLINE, Layer: 0, Color: 256
Type: CIRCLE, Layer: 0, Color: 256
Type: CIRCLE, Layer: 0, Color: 256
Type: CIRCLE, Layer: 0, Color: 256
Type: CIRCLE, Layer: 0, Color: 256
Type: CIRCLE, Layer: 0, Color: 256
Type: CIRCLE, Layer: 0, Color: 256
No dimensions found.
Details for /content/drive/My Drive/modified_e5.dxf:
Type: POLYLINE, Layer: 0, Color: 256
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
No dimensions found.
Details for /content/drive/My Drive/annotated_e5.dxf:
Type: POLYLINE, Layer: 0, Color: 256
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
Type: CIRCLE, Layer: ModifiedLayer, Color: 4
Type: CIRCLE, Layer: ModifiedLayer, 